In [1]:
from ncn.evaluation import Evaluator
from ncn.data import get_datasets

In [2]:
path_to_weights = "/home/timo/Downloads/models/NCN_8_13_11_GRU_1_256_512.pt"
path_to_data = "/home/timo/DataSets/KD_arxiv_CS/arxiv_data.csv"
data = get_datasets(path_to_data)

INFO:ncn.data:Getting fields...
INFO:ncn.data:Loading dataset...
INFO:ncn.data:Building vocab...


In [3]:
evaluator = Evaluator([4,4,5,6,7], [1,2], 512, 256, path_to_weights, data, evaluate=True, show_attention=False)

/home/timo/Programs/miniconda3/envs/KD/lib/python3.7/site-packages/torch/nn/modules/rnn.py:54: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
INFO:ncn.evaluation:INITIALIZING NEURAL CITATION NETWORK WITH AUTHORS = True
Running on: cpu
Number of model parameters: 76,821,300
Encoders: # Filters = 512, Context filter length = [4, 4, 5, 6, 7],  Context filter length = [1, 2]
Embeddings: Dimension = 256, Pad index = 1, Context vocab = 30002, Author vocab = 30002, Title vocab = 30004
Decoder: # GRU cells = 1, Hidden size = 512
Parameters: Dropout = 0.2, Show attention = True
-------------------------------------------------
INFO:ncn.evaluation:Creating corpus in eval=True mode.
INFO:ncn.evaluation:Number of samples in BM25 corpus: 50235


In [4]:
at_5 = evaluator.recall(5)
print(at_5)

ImportError: IntProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html